In [ ]:
import numpy as np
import pandas as pd

from urllib.parse import urlparse, parse_qs

In [ ]:
# Set variables
# Though a wee bit improper I'm  defining this outside the function in the event I later pull variables out
# into a seperate config file
_channel_dict = {
    'crm': 'Offline',
    'email': 'Email',
    'facebook': 'Paid Social',
    'fbap': 'Paid Social',
    'google': 'Organic Search',
    'googlesearch': 'Paid Search',
    'hs_automation': 'Email',
    'hs_email': 'Email',
    'instagram': 'Organic Social',
    'linkedin': 'Organic Social',
    'linktree': 'Paid Social',
    'referral': 'Referral',
    'twitter': 'Organic Social',
    'youtube': 'Referral',
    'Unknown': 'Direct',
    'undefined': 'Direct'
}

In [ ]:
def extract_utms(_df, _column=str):
    _utm_list = ['utm_campaign', 'utm_source', 'utm_medium', 'utm_term', 'utm_content']
    global _channel_dict
    i = 0
    _df = _df.fillna('')

    for _url in _df[_column]:
        _parsed_url = urlparse(_url)
        _queries_dict = parse_qs(_parsed_url.query)
        
        for _utm in _utm_list:
            _value = _queries_dict.get(_utm)
            if _value != None:
                _a, *_b = _value
                _df.loc[i, _utm] = _a
            else:
                _a = 'Unknown'
                _df.loc[i, _utm] = _a
        
        i += 1
        
    _df['Original Channel'] = _df['utm_source'].apply(lambda x: _channel_dict.get(x))
    _df['Original Channel'] = np.where((_df['utm_medium'].str.contains('organic')) & (_df['utm_source'] == 'facebook'), 
                                       'Organic Social', 
                                       _df['Original Channel'])
        
    return _df

In [ ]:
origins_df = pd.read_csv(r'sales-lead-source.csv', error_bad_lines=False)

In [ ]:
origins_df

In [ ]:
sourced_df = extract_utms(origins_df, 'First Page Seen')

In [ ]:
sourced_df

In [ ]:
sourced_df.groupby('utm_source').count().sort_values('Contact ID', ascending=False)[:20]

In [ ]:
sourced_df['Create Date'] = sourced_df['Create Date'].apply(lambda x: pd.to_datetime(x).date())

In [ ]:
pd.crosstab(sourced_df['Create Date'], sourced_df['utm_source']).to_csv(r'sales-lead-source.csv')